## Notes

As 'area of influence' we define 
1. The Influencer's network of engaged users(users that retweet)
2. The geographic area of the users, belonging to the Influencer's network
So the analysis will include both definitions and both graphs

# [Stage 1] Data acquisition

### Initialization of nessesary libraries and tools

Import of necessary libraries and definition of necessary keys for Twitter API

In [1]:
#Core Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx


import math #math.log() used in visualisations
import tweepy #Twitter API Wrapper library
import time #Sleep functionality to keep up with twitter's Quotas
import random #Contribute with time library to keep the twitter quotas under limit


In [2]:

API_KEY = 'yNrkYGayTA4tqIRWfNmwd350n'
API_SECRET_KEY = 'MA8EOw0hJp55gjWrwupYjOdmuTRxBvgpaal7lycwobNAqnEckx'
ACCESS_TOKEN = '1342791046317670400-AeXR52Ohn483w4wzQQ6Zgwzef8Cx3T'
ACCESS_TOKEN_SECRET = 'g1W4dKjZrFlHlvkMOWLlBQ7PtcIAFpRAgm4gaoYMoAfDg'
BEARER_TOKEN = 'AAAAAAAAAAAAAAAAAAAAAExyUgEAAAAAdgqhXDb9kFhloRV35jEE50HTeUY%3DXRvwQyJDaYvYHM5NHHWPpCDxUt7CsmJfdoAQHPA9m0SKQL3dY6'


Initialization of tweepy library

In [3]:

auth = tweepy.OAuthHandler(API_KEY, API_SECRET_KEY)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
api = tweepy.API(auth) #Client to be used for twitter's V1 API calls
client=tweepy.Client(BEARER_TOKEN,API_KEY,API_SECRET_KEY,ACCESS_TOKEN,ACCESS_TOKEN_SECRET) #Client to be used for twitter's V2 API calls


The First operation to be done, is to retrieve for our choosen influencers their unique ID's

In [4]:
billId=str(api.get_user(screen_name='BillGates')._json['id'])
elonId=str(api.get_user(screen_name='elonmusk')._json['id'])
ellenId=str(api.get_user(screen_name='TheEllenShow')._json['id'])


In [5]:
[billId,elonId,ellenId] #Their retrieved ID's

['50393960', '44196397', '15846407']

### Tweets retrieval 

In this section, we will retrieve the maximum allowed number of tweets, 3100 per user, because the maximum allowed retrieved tweets per request are 100, we are forced to use pagination

From the following data retrieval, we are interested only in the network of each influencer, therefore we drop all of the columns but the tweet_id for the given user_id

In [6]:
userTweetMapFrame = pd.DataFrame(columns=['user_id','tweet_id']) #The DataFrame that will hold the tweets
max_results=100 #Maximum allowed number of tweets per request = 100

#Other variables
userIds=[billId,elonId,ellenId]
paginatorSelector=0 

In [7]:
#We will retrieve 100*31 = 3100 tweets for every influencer of choice(limit=31)
billPaginator = tweepy.Paginator(client.get_users_tweets, id=billId,max_results=max_results,limit=31)
elonPaginator = tweepy.Paginator(client.get_users_tweets, id=elonId,max_results=max_results,limit=31)
ellenPaginator = tweepy.Paginator(client.get_users_tweets, id=ellenId,max_results=max_results,limit=31)
paginators = [billPaginator,elonPaginator,ellenPaginator]


In [8]:
#The sole purpose of this function is to enclose the following, time-intensive code that needs a over 20
#minutes to complete, in order to be able to run this analysis top-to-bottom in a reasonable amount of time,
#the code can be run at any time by uncommenting the call below(indicated with ********* (asterisks)), The result 
#of this computation can be retrieved at any time by reading the file tweetMapFull.csv

def getAllAvailableTweets():
    userTweetMapFrame = pd.DataFrame(columns=['user_id','tweet_id'])
    for batchOfTweets in billPaginator:
        try:
            series = map(lambda x:x.id,batchOfTweets.data)
            ids=[userIds[0]]*len(batchOfTweets.data)
            frame = pd.DataFrame({'user_id':ids,'tweet_id':series})
            print(frame)
            userTweetMapFrame=userTweetMapFrame.append(frame)
            userTweetMapFrame.to_csv("tweetMapFull.csv",index=False)
            time.sleep(2)
        except:
            continue

    for batchOfTweets in elonPaginator:
        try:
            series = map(lambda x:x.id,batchOfTweets.data)
            ids=[userIds[1]]*len(batchOfTweets.data)
            frame = pd.DataFrame({'user_id':ids,'tweet_id':series})
            print(frame)
            userTweetMapFrame=userTweetMapFrame.append(frame)
            userTweetMapFrame.to_csv("tweetMapFull.csv",index=False)
            time.sleep(2)
        except:
            continue

    for batchOfTweets in ellenPaginator:
        try:
            series = map(lambda x:x.id,batchOfTweets.data)
            ids=[userIds[2]]*len(batchOfTweets.data)
            frame = pd.DataFrame({'user_id':ids,'tweet_id':series})
            print(frame)
            userTweetMapFrame=userTweetMapFrame.append(frame)
            userTweetMapFrame.to_csv("tweetMapFull.csv",index=False)
            time.sleep(2)
        except:
            continue
            
#getAllAvailableTweets() #*********
#Checkpoint 1
tweetMapFull = pd.read_csv("tweetMapFull.csv",dtype = {'user_id':str,'tweet_id':str})

In [9]:
tweetMapFull #Example of the retrieved data

,user_id,tweet_id
0,50393960,1464659614448771073
1,50393960,1464288045541199874
2,50393960,1463528104614236167
3,50393960,1463186751619436546
4,50393960,1462583003272474628
...,...,...
6494,15846407,1182026314636619776
6495,15846407,1182017738627108865
6496,15846407,1182016640537354240
6497,15846407,1182000506132021249


6k retweets seems a suspicious number, given that we have 3 influencers and have taken 3100 tweets from each of them, the number should be closer to 9k, lets have a look at how many tweets we have per user

In [10]:
tweetMapFull.groupby(by='user_id').size()

user_id
15846407    3099
44196397     300
50393960    3100
dtype: int64

In [11]:
[billId,elonId,ellenId] #Their retrieved ID's

['50393960', '44196397', '15846407']

Elon has only 300 tweets, bill and ellen has thousands, to not bias our analysis we keep 300 tweets on each account


In [12]:
billTweets = tweetMapFull.loc[tweetMapFull['user_id']==billId][0:300]
elonTweets = tweetMapFull.loc[tweetMapFull['user_id']==elonId][0:300]
ellenTweets = tweetMapFull.loc[tweetMapFull['user_id']==ellenId][0:300]

In [13]:
#Checkpoint 2
finalExport = billTweets.append(elonTweets).append(ellenTweets)
finalExport.to_csv("tweetMap.csv",index=False)
finalExport

,user_id,tweet_id
0,50393960,1464659614448771073
1,50393960,1464288045541199874
2,50393960,1463528104614236167
3,50393960,1463186751619436546
4,50393960,1462583003272474628
...,...,...
3695,15846407,1433245716202209283
3696,15846407,1433242268459094020
3697,15846407,1433239858735026179
3698,15846407,1433234633320136705


## Retweets and user locations retrieval

In the following code, we will retrieve as many retweeter accounts as possible for those 900 tweets(300 per influencer). We will also request from Twitter via the metadata, the location field of the particular user who performed the retweet. The Location field will be used for the geolocation area of influence analysis later on

In [14]:
influencer_tweets=pd.read_csv("tweetMap.csv",dtype = {'user_id':str,'tweet_id':str})

In [15]:
tweets_retweeters = pd.DataFrame(columns=['tweet_id','retweeter_id','retweeter_location']) #Result DataFrame
iterator=iter(influencer_tweets['tweet_id'].tolist())  #Iterator declared here, in order the following code to 
#be fault-tolerant (if the function below stops, the state of the iterator remains in memory)

In [16]:

#The sole purpose of this function is to enclose the following, time-intensive code that needs a over 12
#hours to complete, in order to be able to run this analysis top-to-bottom in a reasonable amount of time,
#the code can be run at any time by uncommenting the call below(indicated with ********* (asterisks)), The result 
#of this computation can be retrieved at any time by reading the file retweetMap.csv

def getAllAvailableReTweets():
    tweets_retweeters= pd.DataFrame(columns=['tweet_id','retweeter_id','retweeter_location'])
    while(True):
        every_influencer_tweet = next(iterator)
        try:
            time.sleep(15) #Time delay to avoid quota
            retweeters = list(client.get_retweeters(every_influencer_tweet,user_fields=['location']).data)
            instances=len(retweeters)
            ids = (map(lambda x:x.id,retweeters))
            locations = (map(lambda x:x.location,retweeters))
            tweets_retweeters_current = pd.DataFrame({
                'tweet_id':[every_influencer_tweet]*instances,
                'retweeter_id':ids,
                'retweeter_location':locations})
            tweets_retweeters=tweets_retweeters.append(tweets_retweeters_current)
            tweets_retweeters.to_csv("retweetMap.csv",index=False)
            print(tweets_retweeters_current)
        except Exception as e:
            print("Ignoring error, probably quota limit, continue"+str(e))
            time.sleep(10) #Further time delay to avoid quota limit 


#getAllAvailableReTweets()*********
#Checkpoint 3
retweets = pd.read_csv("retweetMap.csv",dtype = {'tweet_id':str,'retweeter_id':str,'retweeter_location':str})
retweets

,Unnamed: 0,tweet_id,retweeter_id,retweeter_location
0,0,1461380998743015425,861629679886880768,"Dallas, TX"
1,1,1461380998743015425,1034572631025098754,El Mundo
2,2,1461380998743015425,1425487033309581314,Bristol 🇪🇺 🇬🇧
3,3,1461380998743015425,1409329636954148869,"Surrey, British Columbia"
4,4,1461380998743015425,10221,philly
...,...,...,...,...
35781,39414,1415752955152666627,1412392837916864514,bead.99@icloud.com
35782,39415,1415752955152666627,2399137929,tulungagung
35783,39416,1415752955152666627,113410319,Brasil
35784,39417,1415752955152666627,751724810640785408,"Karachi, Pakistan"


## Convert vague locations into real countries

In [17]:
#Loading from Checkpoint 3
tweets =pd.read_csv("tweetMap.csv",dtype = {'user_id':str,'tweet_id':str})
retweets = pd.read_csv("retweetMap.csv",dtype = {'tweet_id':str,'retweeter_id':str,'retweeter_location':str})

In [18]:
#Eliminate entries with unknown location
retweets['retweeter_location']=retweets['retweeter_location'].map(lambda x:str(x))
retweets=retweets.loc[retweets['retweeter_location']!='nan']
retweets

,Unnamed: 0,tweet_id,retweeter_id,retweeter_location
0,0,1461380998743015425,861629679886880768,"Dallas, TX"
1,1,1461380998743015425,1034572631025098754,El Mundo
2,2,1461380998743015425,1425487033309581314,Bristol 🇪🇺 🇬🇧
3,3,1461380998743015425,1409329636954148869,"Surrey, British Columbia"
4,4,1461380998743015425,10221,philly
...,...,...,...,...
35781,39414,1415752955152666627,1412392837916864514,bead.99@icloud.com
35782,39415,1415752955152666627,2399137929,tulungagung
35783,39416,1415752955152666627,113410319,Brasil
35784,39417,1415752955152666627,751724810640785408,"Karachi, Pakistan"


Any attempt to analyse the area of influence for the given account will be in vain, if we don't have standardized Data points for locations, the Location field of Twitter seems to be an non-validated field that accepts invalid locations

In [19]:
retweets.loc[retweets['retweeter_id']=='1253442333334548482']

,Unnamed: 0,tweet_id,retweeter_id,retweeter_location
6013,6013,1380231432006631425,1253442333334548482,Saving the world
6815,6815,1367519920393654274,1253442333334548482,Saving the world


We need a process of converting these unreliable data points into standard countries if possible and discarding the rest, The following is the algorithm to perform such a task

To construct such an algorithm, we will perform string matching on the data points, trying to extract names of cities, areas or states, finding the country that contains the city/state in question. Given the fact of multiple cities around the world that share similar names, we will assign on each finding a weight proportional to the population of the country in question.

In [20]:
#Csv Containing ~150k datapoints, of towns and their states/countries
#The data, originally as a .sql file, taken from the following Github project 
#https://github.com/dr5hn/countries-states-cities-database
#And exported to .csv using a custom script and a JOIN statement
world = pd.read_csv("places_of_the_world.csv") 
world

,#,country_name,state_name,city_name
0,1,Afghanistan,Ghazni,Ghazni
1,2,Afghanistan,Badghis,Ghormach
2,3,Afghanistan,Badghis,Qala i Naw
3,4,Afghanistan,Bamyan,Bāmyān
4,5,Afghanistan,Bamyan,Panjāb
...,...,...,...,...
147961,147962,Zimbabwe,Masvingo Province,Mashava
147962,147963,Zimbabwe,Masvingo Province,Masvingo
147963,147964,Zimbabwe,Masvingo Province,Masvingo District
147964,147965,Zimbabwe,Masvingo Province,Mwenezi District


In [34]:
#Saved Html page, containing the population of each country in question
#Original URL : https://www.worldometers.info/world-population/population-by-country/
populations = pd.read_html('../populations/populations.html')[0]
populations=populations[['Country (or dependency)','Population (2020)']]
populations

,Country (or dependency),Population (2020)
0,China,1439323776
1,India,1380004385
2,United States,331002651
3,Indonesia,273523615
4,Pakistan,220892340
...,...,...
230,Montserrat,4992
231,Falkland Islands,3480
232,Niue,1626
233,Tokelau,1357


In [22]:
def getCountryWeight(country:str):
    """
    This simple function calculates and returns the weight of each country by searching on the 
    aforementioned countries table
    """
    try:
        populationOfCountry = populations.loc[populations['Country (or dependency)']==country]
        return int(populationOfCountry['Population (2020)'])
    except:
        return 0

In [23]:
def translateToCountry(vagueLocation,world):
    """
    This simple function is the core of our translation. Splits the vague location in question into smaller 
    tokens (given some criteria, see below) and searches for occurrences of those tokens in the 
    places_of_the_world.csv. If multiple occurrences are found, it assigns a weight on each and returns the one with
    the biggest weight, or 'None' if there is no hit. The code is quite slow (O^2) and ideally, the search should be
    performed using some sort of a hashmap for better results, but given the time constraints, it wasn't possible to find 
    a faster solution, but a faster solution is possible
    """

    #Step one, split location into tokens
    if("," in vagueLocation):
        #Some locations is like 'Chinatown, New York' , containing comma
        elements=list(map(lambda x:x.strip().lower(),vagueLocation.split(",")))
    elif("_" in vagueLocation):
        #Some other locations, containing underscore
        elements=list(map(lambda x:x.strip().lower(),vagueLocation.split("_")))
    else:
        #If none of the aforementioned exist, then split by empty character
        #If the location does not contain any of the following (' ',',','_') then
        #A single-token-list will be produced (i.e 'Canada' = ['Canada'])
        elements=list(filter(lambda x:len(x)>1,vagueLocation.split(" ")))
        elements=list(map(lambda x:x.lower(),elements))
        
    #If Empty, short-circut
    if(len(elements)==0):
        return 'None'
        
    
    countries=[] #List to be populated with pairs of [country_found,weight_of_country_found]
    for index,row in world.iterrows():
        
        #if Country name or State name exists, the short_circut and return the country
        if(row['country_name'].lower() in elements or row['state_name'].lower() in elements):
            return row['country_name']
        #If city name found, add the country name that belongs to to the countries list
        elif(row['city_name'].lower() in elements):
            countries.append((row['country_name'],getCountryWeight(row['country_name'])))
            
    #sort the countries based on the weight, the biggest is not on top
    countries.sort(reverse=True,key=lambda x:x[1])
    
    #If not hits found, return none, otherwise return the country with the biggest weight
    if(len(countries)==0):
        return 'None'
    return(countries[0][0])

def passAndPrint(vague):
    """
    Simple function that used for logging during the computation
    """
    ans=translateToCountry(vague,world)
    print("vague "+str(vague)+" = true "+str(ans))
    return ans

In [24]:
#The sole purpose of this function is to enclose the following, time-intensive code that needs a over 12
#hours to complete, in order to be able to run this analysis top-to-bottom in a reasonable amount of time,
#the code can be run at any time by uncommenting the call below(indicated with ********* (asterisks)), The result 
#of this computation can be retrieved at any time by reading the file retweetsMapRealLocations.csv

def convertVagueLocationsToCountries():
    """
    This function 
    """
    retweets['retweeter_location']=retweets['retweeter_location'].map(lambda x:passAndPrint(x))
    retweets.to_csv("retweetsMapRealLocations.csv",index=False)
    

#convertVagueLocationsToCountries()*********

#Checkpoint 4
retweetsMapRealLocations=pd.read_csv("retweetsMapRealLocations.csv",dtype = {'tweet_id':str,'retweeter_id':str,'retweeter_location':str})
retweetsMapRealLocations

,Unnamed: 0,tweet_id,retweeter_id,retweeter_location
0,0,1461380998743015425,861629679886880768,United States
1,1,1461380998743015425,1034572631025098754,None
2,2,1461380998743015425,1425487033309581314,United States
3,3,1461380998743015425,1409329636954148869,Canada
4,4,1461380998743015425,10221,None
...,...,...,...,...
35781,39414,1415752955152666627,1412392837916864514,None
35782,39415,1415752955152666627,2399137929,Indonesia
35783,39416,1415752955152666627,113410319,None
35784,39417,1415752955152666627,751724810640785408,Pakistan


In [25]:
#Dropping all the datapoints that the previous algorithm failed to recognise and ascosiate with a real country
retweetsMapRealLocationsDiscardNones=retweetsMapRealLocations.loc[retweetsMapRealLocations['retweeter_location']!='None']
retweetsMapRealLocationsDiscardNones.to_csv("retweetsMapRealLocationsDiscardNones.csv",index=False)
retweetsMapRealLocationsDiscardNones

,Unnamed: 0,tweet_id,retweeter_id,retweeter_location
0,0,1461380998743015425,861629679886880768,United States
2,2,1461380998743015425,1425487033309581314,United States
3,3,1461380998743015425,1409329636954148869,Canada
5,5,1461380998743015425,1053386113199603712,Brazil
6,6,1461380998743015425,38105260,Australia
...,...,...,...,...
35778,39411,1415752955152666627,984581120271331329,Australia
35779,39412,1415752955152666627,172579237,India
35782,39415,1415752955152666627,2399137929,Indonesia
35784,39417,1415752955152666627,751724810640785408,Pakistan


## Final Merging of the data

We will now merge our acquired data into a unified DataFrame

In [26]:
"""
Merge tweetMap with retweetsMapRealLocations
plot
"""
combined=tweets.merge(retweetsMapRealLocationsDiscardNones,on="tweet_id")
bill = combined.loc[combined['user_id']==billId]
elon = combined.loc[combined['user_id']==elonId]
ellen= combined.loc[combined['user_id']==ellenId]
countries = combined['retweeter_location'].unique()

combined

,user_id,tweet_id,Unnamed: 0,retweeter_id,retweeter_location
0,50393960,1461380998743015425,0,861629679886880768,United States
1,50393960,1461380998743015425,2,1425487033309581314,United States
2,50393960,1461380998743015425,3,1409329636954148869,Canada
3,50393960,1461380998743015425,5,1053386113199603712,Brazil
4,50393960,1461380998743015425,6,38105260,Australia
...,...,...,...,...,...
21922,15846407,1433188250940702721,34889,1260292359239036928,Taiwan
21923,15846407,1433188250940702721,34890,1226492046460014593,Canada
21924,15846407,1433188250940702721,34891,1337887775891337218,United States
21925,15846407,1433188250940702721,34892,820879792660762624,United States


# [Stage 2] Visualisation of the Aquired Data


After acquiring all the necessary data, the time is come to start producing visualisations and getting an insight
on the networks generated

## Account's Influence network analysis
In this section we will use our data to investigate the topology of the influencer's retweeters networks, getting some useful insights on how ideas are spread on the Twitter community

In [27]:
#Calculation of the weight of each influencer. Each influencer's weight is proportional to the amount of the retweets
#on the given account. If x has retweeted 16 bill gates tweets their weight will be 16.
weights=retweetsMapRealLocationsDiscardNones.groupby(by=["retweeter_id"]).size().reset_index()
weights['weight']=weights[0]
del weights[0]
weights

,retweeter_id,weight
0,1000190726326685698,1
1,1000218707455553537,1
2,1000245113518309376,1
3,1000323282070056960,1
4,1000368846790770688,1
...,...,...
13039,999514335423561728,4
13040,99959293,1
13041,999647495423639552,1
13042,999731945625006080,1


In [28]:
#Creation of the edge list
RetweeterDensityEdgeList = pd.DataFrame({'user_id':combined['user_id'],'retweeter_id':combined['retweeter_id']})
RetweeterDensityEdgeList=RetweeterDensityEdgeList.merge(weights,on='retweeter_id')

In [29]:
#Example of the edge list with weights(10 records)
RetweeterDensityEdgeList[1:10]

,user_id,retweeter_id,weight
1,50393960,861629679886880768,8
2,50393960,861629679886880768,8
3,50393960,861629679886880768,8
4,50393960,861629679886880768,8
5,50393960,861629679886880768,8
6,50393960,861629679886880768,8
7,50393960,861629679886880768,8
8,50393960,1425487033309581314,2
9,50393960,1425487033309581314,2


In [30]:
def drawRetweeterDensity(edge_list):
    """
    This function accepts the edge_list of the influence network and plots it on screen
    Returns a list of statistics for the produced graph
    """
    areaOfInfluenceGraph= nx.Graph()
    pos = nx.spring_layout(areaOfInfluenceGraph)
    for index, row in edge_list.iterrows():
        areaOfInfluenceGraph.add_edges_from([ (row['user_id'],row['retweeter_id'])], weight=row['weight'])
    pos = nx.spring_layout(areaOfInfluenceGraph)
    plt.figure(3,figsize=(150,150)) 
    nx.draw(areaOfInfluenceGraph,pos,with_labels=False)
    plt.show() 
    return [
        nx.degree_histogram(areaOfInfluenceGraph),
        nx.clustering(areaOfInfluenceGraph),
        nx.betweenness_centrality(areaOfInfluenceGraph),
        nx.degree_assortativity_coefficient(areaOfInfluenceGraph)]

In [31]:
def drawDegreeDistributionHistogram(degree_hist):
    plt.bar(range(0,len(degree_hist)),degree_hist)
    plt.xlabel("Degree")
    plt.ylabel("Frequency")
    plt.show() 
def drawDegreeDistributionHistogramFiltered(degree_hist):
    k,v = zip(*degree_hist)
    plt.bar(k,v)
    plt.xlabel("Degree")
    plt.ylabel("Frequency")
    plt.show() 

In [32]:
stats = drawRetweeterDensity(RetweeterDensityEdgeList)

KeyboardInterrupt: 

## Network Statistics
In this section, we will investigate the topology and the statistics of the aforementioned network

The produced network has a very interesting topology indeed, it shows how information flows between communities of different people. There are clusters of people sharing common ideas but, with the help of a few retweeters that retweet content from more than one account, the area of influence is significantly increased. Evidence suggests that in most real-world networks, as well as social networks, nodes tend to create tightly knit groups characterised by a relatively high density of ties(Holland and Leinhardt, 1971,Watts and Strogatz, 1998), something that we can confirm by our networks topology. Let's investigate the topology with the help of statistics to extract insightful knowledge.

### Degree Distribution

In [ ]:
#Aquire the degree distibution
degree_distribution = stats[0]

Due to the nature of the network in question, the majority of the nodes have 0 connectivity, this shadows any meaningful information from the degree distribution histogram, as seen below

In [ ]:
drawDegreeDistributionHistogram(degree_distribution)

We need a way to expose the nature of the network distribution histogram, for this, we will perform the following operations
1. Omit the degrees occurrences when degree=0
2. plot the logarithm of the frequency and the degree

In [ ]:

zipped=list(zip(degree_distribution,range(0,len(degree_distribution))))
zipped_filtered = list(filter(lambda x:x[0]!=0,zipped))
zipped_filtered_logged = list(map(lambda x:(math.log(x[0]),math.log(x[1])),zipped_filtered))
drawDegreeDistributionHistogramFiltered(zipped_filtered_logged)

In [ ]:
zipped_filtered

We can now reveal the true nature of the histogram, we see that the majority of the nodes (>99%) have connectivity=1 (only interacted with the influencer in question), a few nodes that interacted with 2 influencers and only one(!!) interacted with all of 3 influencers in question. Finally, the 3 remaining nodes are the influencer nodes themselves.

## Cluster Coefficient

In [ ]:
cluster_coeff = stats[1] #Get Clustering coeficcient statistic

The cluster coefficient of this network is zero, as every node has zero clustering coefficient, that's because of  the topology of the network itself, as there is no clusters formed (3 star topologies loosely connected)

In [ ]:
plt.hist(cluster_coeff.values(),bins=8)

In [ ]:
#There is not a single node with cluster coefficient other than 0
series_cluster_coeff = pd.Series(cluster_coeff.values()).reset_index()
series_cluster_coeff.loc[series_cluster_coeff[0]!=0] 

## Betweenness Centrality

Given the topology of the network, and combining the information taken from the degree distribution, we can infer that
1. There are 90 nodes that retweet from more than 1 account
2. There is 1 node retweeting from all 3 accounts selected
3. Therefore, there are 91(common) + 3(3 influencer nodes) nodes with a centrality of more than 0

Let's have a look and verify our findings


In [ ]:
betweenness_centrality=pd.Series(stats[2]).reset_index()
important_nodes = betweenness_centrality.loc[betweenness_centrality[0]>0.0]
print(important_nodes)
print(len(important_nodes) == 91 + 3)

## Ascosiativity

As expected, due to the nature of the network topology (>99% of nodes, connected with only one node) the network associativity expected to be extremely high

In [ ]:
ascosiativity=pd.Series(stats[3]).reset_index()
ascosiativity

## Geographic Influence 
In this section we will use our data to extract intuition on the geographic area of influence

In the first attempt, we will draw our graph between each influencer and the country, Each country distance from the account node will be proportional of the number of unique retweets coming from this country.

In [ ]:
#Calculation of the edge list with weights for the geographic area of influence map
countryInfluence1 = combined.groupby(by=['user_id','retweeter_location']).size().reset_index()
countryInfluence1['weight'] = countryInfluence1[0]
del countryInfluence1[0]

In [ ]:
#Example of the edge list with weights(10 records)
countryInfluence1[1:10]

In [ ]:
#Draw central account and retweeterIDs, the distance from center is how many retweets by specific user
def drawAccountInfluence1(account,n):
    """
    This function accepts the edge_list of the Geographic Influence network and plots 
    """
    areaOfInfluenceGraph= nx.Graph()
    pos = nx.spring_layout(areaOfInfluenceGraph)
    for index, row in account.iterrows():
        areaOfInfluenceGraph.add_edges_from([ (row['user_id'],row['retweeter_location'])], weight=row['weight'])
        areaOfInfluenceGraph.add_edges_from([ (row['retweeter_location'],row['user_id'])], weight=row['weight'])
    pos = nx.spring_layout(areaOfInfluenceGraph)
    plt.figure(3,figsize=(60,60)) 
    nx.draw(areaOfInfluenceGraph,pos,with_labels=True)
    plt.show() 

In [ ]:
drawAccountInfluence1(countryInfluence1,countryInfluence1.size)

## Geographic Influence improved
The previous approach had a core issue, it didn't account for engagement. Engagement quantifies how much user interaction there is with your campaign ([Source](https://business.twitter.com/en/help/overview/twitter-ads-glossary.html)). There are two statistics that we can count on


1. Number of Unique retweets per account per country.
2. Number of Unique retweeters per account per country.

If we account only for the first parameter(as we did on the previous graph) then a country A with 100 retweets coming from 100 retweeters(accounts) will be equal to a country B with 100 retweets from 30 retweeters(accounts). Obviously, the country B is way more engaged

We define our weight function as... $$f(u,r) = 1-\frac{u}{r}$$
Where 
1. u: Users engaged with this account in the given country
2. r: Retweets produced by those users for the given country

In [ ]:
uniqueUsers = combined.groupby(by=['user_id','tweet_id','retweeter_location']).size().reset_index()
uniqueRetweets = combined.groupby(by=['user_id','retweeter_location']).size().reset_index()

Calculation of the edge list for the improved geographic location graph

In [ ]:
#Calculation for bill gates 

billUsers = uniqueUsers.loc[uniqueUsers['user_id']==billId].groupby(by=['retweeter_location']).size().reset_index()
billRetweets = uniqueRetweets.loc[uniqueRetweets['user_id']==billId].reset_index()

del billRetweets['index']
billUsers['unique_users'] = billUsers[0]
del billUsers[0]
billRetweets['unique_tweets']=billRetweets[0]
del billRetweets[0]

mergedBill = billUsers.merge(billRetweets,on='retweeter_location')
mergedBill['score']=1-mergedBill['unique_users']/mergedBill['unique_tweets']

#Example, Score of United States
mergedBill.loc[mergedBill['retweeter_location']=='United States']

In [ ]:
#Calculation for elon musk

elonUsers = uniqueUsers.loc[uniqueUsers['user_id']==elonId].groupby(by=['retweeter_location']).size().reset_index()
elonRetweets = uniqueRetweets.loc[uniqueRetweets['user_id']==elonId].reset_index()


del elonRetweets['index']
elonUsers['unique_users'] = elonUsers[0]
del elonUsers[0]
elonRetweets['unique_tweets']=elonRetweets[0]
del elonRetweets[0]

mergedElon = elonUsers.merge(elonRetweets,on='retweeter_location')
mergedElon['score']=1-mergedElon['unique_users']/mergedElon['unique_tweets']

#Example, Score of United States
mergedElon.loc[mergedElon['retweeter_location']=='United States']

In [ ]:
#Calculation for ellen DeGenerous

ellenUsers = uniqueUsers.loc[uniqueUsers['user_id']==ellenId].groupby(by=['retweeter_location']).size().reset_index()
ellenRetweets = uniqueRetweets.loc[uniqueRetweets['user_id']==ellenId].reset_index()

del ellenRetweets['index']
ellenUsers['unique_users'] = ellenUsers[0]
del ellenUsers[0]
ellenRetweets['unique_tweets']=ellenRetweets[0]
del ellenRetweets[0]

mergedEllen = ellenUsers.merge(ellenRetweets,on='retweeter_location')
mergedEllen['score']=1-mergedEllen['unique_users']/mergedEllen['unique_tweets']

#Example, Score of United States
mergedEllen.loc[mergedEllen['retweeter_location']=='United States']

In [ ]:
#Merged edge list with weights
merged=mergedBill.append(mergedElon).append(mergedEllen)

In [ ]:
#Example of edge list with weights(10 records)
merged[1:10]

In [ ]:

def drawEngagement(account,color_map=None):
    """
    This function accepts the edge_list of the improvement Geographic Influence network and plots 
    """
    color_map= ["green"]*(len(account['user_id'].unique())+len(account['retweeter_location'].unique())) if color_map==None else color_map 
    areaOfInfluenceGraph= nx.Graph()
    pos = nx.spring_layout(areaOfInfluenceGraph)
    for index, row in account.iterrows():
        areaOfInfluenceGraph.add_edge(row['user_id'],row['retweeter_location'], weight=row['score'])
        areaOfInfluenceGraph.add_edge(row['retweeter_location'],row['user_id'], weight=row['score'])
    pos = nx.spring_layout(areaOfInfluenceGraph)
    plt.figure(3,figsize=(50,50)) 
    nx.draw(areaOfInfluenceGraph,pos,with_labels=True,node_color=color_map)
    plt.show() 
    return [
        nx.degree_histogram(areaOfInfluenceGraph),
        nx.clustering(areaOfInfluenceGraph),
        nx.betweenness_centrality(areaOfInfluenceGraph),
        nx.degree_assortativity_coefficient(areaOfInfluenceGraph)]


In [ ]:
stats=drawEngagement(merged)

## Network Statistics

The network produced, contains similar characteristics to the previous network, providing however an insight view on the influenced geographic areas. Let's have a quick look at the statistics of this network

## Degree Distribution

In [ ]:
#Aquire the degree distibution
degree_distribution = stats[0]

In [ ]:
drawDegreeDistributionHistogram(degree_distribution)

We can see that the majority of the countries have very low engagement scores, with very few having an extremely high score(closer to the centre), some of the countries are shared between the accounts and some others not

In [ ]:
high_engaged_countries=merged.loc[merged['score']>0.6]
low_engaged_countries=merged.loc[merged['score']<=0.6]


In [ ]:
high_engaged_countries

In [ ]:
low_engaged_countries

## Betweenness Centrality

The Betweenness Centrality of this network is trivial, due to the topology, there are 3 central account nodes(influencer's nodes) that have very high betweenness centrality score, the rest of the nodes contain near zero score 

In [ ]:
betweenness_centrality=pd.Series(stats[2]).reset_index()
important_nodes = betweenness_centrality.loc[betweenness_centrality[0]>0.1]
not_important_nodes = betweenness_centrality.loc[betweenness_centrality[0]<=0.1]
important_nodes

In [ ]:
not_important_nodes

## Ascosiativity

As expected again, due to the nature of the network topology (>99% of nodes, connected with only one node) the network associativity expected to be extremely high

In [ ]:
ascosiativity=pd.Series(stats[3]).reset_index()
ascosiativity

## Cluster Coefficient

In [ ]:
cluster_coeff = stats[1] #Get Clustering coeficcient statistic

In [ ]:
#There is not a single node with cluster coefficient other than 0
series_cluster_coeff = pd.Series(cluster_coeff.values()).reset_index()
series_cluster_coeff.loc[series_cluster_coeff[0]!=0] 